In [1]:
import nltk
from nltk.stem.porter import PorterStemmer

import numpy as np
import torch
import torch.nn as nn
import random
from torch.utils.data import Dataset, DataLoader
import json

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
stemmer = PorterStemmer()

In [4]:
all_words = []
all_labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        all_words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in all_labels:
        all_labels.append(intent['tag'])

In [5]:
all_words = [stemmer.stem(w.lower()) for w in all_words if w != "?"]
all_words = sorted(list(set(all_words)))


In [6]:
training = []
output = []

out_empty = [0 for _ in range(len(all_labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in all_words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[all_labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [7]:
training = np.array(training, dtype="float32")
output = np.array(output, dtype="float32")


In [8]:
num_epochs = 1000
batch_size = 7
learning_rate = 0.001
input_size = len(training[0])
hidden_size = 8
output_size = len(all_labels)
print(input_size, output_size)

41 7


In [9]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(training)
        self.x_data = training
        self.y_data = output

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [10]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [11]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.1743
Epoch [200/1000], Loss: 0.3836
Epoch [300/1000], Loss: 0.0614
Epoch [400/1000], Loss: 0.0068
Epoch [500/1000], Loss: 0.0001
Epoch [600/1000], Loss: 0.0042
Epoch [700/1000], Loss: 0.0016
Epoch [800/1000], Loss: 0.0009
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0003


In [13]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return torch.tensor(bag, dtype=torch.float, device=device)

In [17]:
tags = [x["tag"] for x in data["intents"]]
sentence = "how old are you"
with torch.no_grad():
    output = model(bag_of_words(sentence, all_words))
    predicted = torch.argmax(output)
    
tag = tags[predicted.item()]
for tg in data["intents"]:
    if tg['tag'] == tag:
        responses = tg['responses']

print(random.choice(responses))

I don't know time runs so fast


In [21]:
data = {
"all_words": all_words,
"tags": tags
}

DATA_FILE = "deploy_sam/data.pth"
torch.save(data, DATA_FILE)

In [ ]:
MODEL_FILE
torch.save(model, PATH)